# Uninformed Search Agent
- Start with a frontier that contains the initial state (**node**, actually).
- Start with an empty explored **set**.
- Repeat:
    - If the frontier **is empty**, then no solution.
    - **Remove a node** from the frontier.
    - If node contains **goal state**, return the solution.
    - Add the node to the **explored set**.
    - **Expand node**, add resulting nodes to the frontier if they aren't already in the frontier or the explored set.
    

In [1]:
class Node:
    """ The Node class is used in the search algorithms.
    It can be trivially extended to include costs
    """
    def __init__(self, state, parent=None, action=None, cost=0):
        """ The constructor needs the current state and action, and a pointer to a
        parent Node.
        For the 'root' node, only the start state.
        """
        self.state = state
        self.parent = parent
        self.action = action
        self.cost = cost
        
    def __str__(self):
        """ Returns a string with the state and action of this node"""
        return f"State: {self.state} Action: {self.action}\n"


In [2]:
class Environment(object):
    """ This class is model specific (not universal at all!)"""
    def __init__(self, maze):
        self.walls = maze[0] # List of "walls" encoded with "ones"
        self.start = maze[1] # tuple with initial coordinates
        self.goal = maze[2] # tuple with goal coordinates
        
    def print(self,solution=None): # Some fancy printing. If solution is provided, it's fancier :-D
        print("Maze:")
        for i, row in enumerate(self.walls):
            for j, col in enumerate(row):
                if col:
                    print("#", end="")
                elif (i, j) == self.start:
                    print("S", end="")
                elif (i, j) == self.goal:
                    print("G", end="")
                elif solution is not None and (i, j) in solution:
                    print("·", end="")
                else:
                    print(" ", end="")
            print()
        print()



In [3]:
class SearchAgent:
    """ Generic class for search algorithms"""
    def __init__(self,environment):
        """ The constructor requires an environment class that, at least contains
        the start and goal states.
        
        If additional information is required, this class can be inherited but you
        should include:
        super().__init__()
        """
        self.frontier = [Node(state=environment.start)] # We play with nodes, not states.
        self.explored = set() # I use a 'set' to avoid repetitions (and order doesn't matter)
        self.goal_state = environment.goal
    
    def print_frontier(self): 
        """ Print the nodes in the frontier"""
        print("Frontier:")
        print("---------")
        for node in self.frontier:
            print(node)
            
    def print_explored(self):
        """ Print the nodes in the explored set"""
        print("Explored:")
        print("---------")
        for node in self.explored:
            print(node)
            
    def is_frontier_empty(self):
        """ A boolean. True if the frontier is empty (so there is no solution)"""
        return len(self.frontier) == 0 # No solution

    def actions(self,node):
        """ Needs to be extended in the child daughter classes of this class.
        This function is the main function codifiying the problem and it has 
        to be specific to the subject at hand.
        """
        pass
    
    def result(self,node):
        """ Needs to be extended in the child daughter classes of this class.
        This function is the main function codifiying the problem and it has 
        to be specific to the subject at hand.
        """
        pass
    
    def remove_node(self):
        """ Needs to be extended in the child daughter classes of this class.
        Here is where we decide if we use a stack, a queue, ...
        """
        pass
    
    def is_goal(self,node):
        """ This function plays two roles:
        1. Check if we have currently reached the goal state.
        2. If so, reconstruct the tree backwards to show the obtained solution.
        """
        if node.state == self.goal_state:
            self.solution = []
            while node.parent is not None:
                self.solution.append(node)
                print(f"{node.state}({node.action})")
                node = node.parent
            raise Exception("Solution found")

    def add_to_frontier(self,node):
        """ Append node to the frontier. """
        self.frontier.append(node)
            
    def add_to_explored(self,node):
        """Add node to explored set (this is a pythonic "set" that's why I use add instead of append."""
        self.explored.add(node) 

    def is_state_in_frontier(self,node):
        """Returns a boolean (True) if the state is in the current frontier"""
        return any(frontier_node.state == node.state for frontier_node in self.frontier)
  
    def is_state_in_explored(self,node):
        """Returns a boolean (True) if the state is in the explored set"""
        return any(explored_node.state == node.state for explored_node in self.explored)
    
    def expand_node(self, node):
        """Here is the main function of the code. We decide RESULTs taking the states and 
        actions into consideration.
        
        I've tried to make this code as versatile as I can. This part should be the same
        for any search problem you program. Cool, uh!
        """
        for child in self.result(node): 
            # Check if the node is not frontier or explored
            if not self.is_state_in_frontier(child) and not self.is_state_in_explored(child):
                self.add_to_frontier(child) # add all the "compatible" children
        
    def solve(self):
        """The main loop. Here is where we implement the "Repeat" part of the algorithm. 
        It follows almost verbatim the description given in the lecture"""
        while not self.is_frontier_empty():
            try:
                node = self.remove_node()
            except:
                print("Solution not found!!")
                
            try:
                self.is_goal(node)
            except:
                return self.solution
            self.add_to_explored(node)
            self.expand_node(node)

In [4]:
class AgentDepthFirst(SearchAgent):
    def remove_node(self):
        return self.frontier.pop() # Remove the last element (this is a stack!!!)
    

class MazeSearch(AgentDepthFirst):
    def __init__(self,environment):
        super().__init__(environment) # Inherit constructor from parent
        self.walls = environment.walls # Store walls to be used in function RESULTS(s,a) below
        
    def actions(self,node):
        row,col = node.state
        possible_actions = [
            ("up",(row-1,col)),
            ("down",(row+1,col)),
            ("left",(row,col-1)),
            ("right",(row,col+1))
        ]
        return possible_actions
        
    def result(self,node):
        children = []
        height = len(self.walls)
        width = len(self.walls[0])
        
        for action, (r,c) in self.actions(node):
            if 0 <= r < height and 0 <= c <width and not self.walls[r][c]: # if not wall or limits
                children.append(Node(state=(r,c),parent=node,action=action))
                
        return children
        
        

In [8]:
maze = ([
[1,1,0,0,0,0,1],# Walls. You can download tons of mazes here https://invpy.com/mazes/, but build a function to convert
[1,1,0,1,1,0,1],
[1,0,0,1,0,0,1],
[1,0,1,1,0,1,1],
[0,0,0,0,0,1,1],
[0,1,1,1,1,1,1]],  
(5,0), # Start state
(1,2)) # Goal state


maze_environment = Environment(maze)
maze_environment.print()

maze_solver = MazeSearch(maze_environment)
maze_solver.print_frontier()
solution_nodes = maze_solver.solve()
solution = [node.state for node in solution_nodes] # To program "print", it's easier to deal with states
maze_environment.print(solution)

Maze:
##    #
##G## #
#  #  #
# ## ##
     ##
S######

Frontier:
---------
State: (5, 0) Action: None

(1, 2)(down)
(0, 2)(left)
(0, 3)(left)
(0, 4)(left)
(0, 5)(up)
(1, 5)(up)
(2, 5)(right)
(2, 4)(up)
(3, 4)(up)
(4, 4)(right)
(4, 3)(right)
(4, 2)(right)
(4, 1)(right)
(4, 0)(up)
Maze:
##····#
##G##·#
#  #··#
# ##·##
·····##
S######



In [9]:
class MazeSearchBFS(MazeSearch):
    def remove_node(self):
        return self.frontier.pop(0) # Remove the first element (this is a queue!)

In [11]:
maze_solver = MazeSearchBFS(maze_environment)
maze_solver.print_frontier()
solution_nodes = maze_solver.solve()
solution = [node.state for node in solution_nodes] # To program "print", it's easier to deal with states
maze_environment.print(solution)

Frontier:
---------
State: (5, 0) Action: None

(1, 2)(up)
(2, 2)(right)
(2, 1)(up)
(3, 1)(up)
(4, 1)(right)
(4, 0)(up)
Maze:
##    #
##G## #
#··#  #
#·## ##
··   ##
S######

